# Mühle GUI

In [ ]:
import ipycanvas
import ipywidgets
import math

In [ ]:
import logging

# create logger with ''
logger = logging.getLogger('GUI')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('log.txt')
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)

In [ ]:
# TODO: create schema to visualize following constants

# Board constants
BOARD_SIZE = 500
CANVAS_PADDING = 30
ROW_WIDTH = 60
PLAYER_PIECE_RADIUS = 12
DEFAULT_PIECE_RADIUS = 5

DEFAULT_STONE_LINE_WIDTH = 1
SELECTED_STONE_LINE_WIDTH = 3

STASH_STONES_SPACING = 5

STASH_HEIGHT = ((PLAYER_PIECE_RADIUS * 2 ) * 2) + (STASH_STONES_SPACING) + (CANVAS_PADDING * 2)
STASH_WIDTH = ((PLAYER_PIECE_RADIUS * 2 ) * 9) + (STASH_STONES_SPACING * 8) + (CANVAS_PADDING * 2)

CANVAS_HEIGHT = BOARD_SIZE
CANVAS_WIDTH = BOARD_SIZE + STASH_WIDTH

STASH_STARTING_POINT_X = BOARD_SIZE
STASH_STARTING_POINT_Y = CANVAS_HEIGHT - STASH_HEIGHT

# text
TEXT_X = BOARD_SIZE + CANVAS_PADDING
TEXT_Y = CANVAS_PADDING
TEXT_MAX_WIDTH = CANVAS_WIDTH - BOARD_SIZE - 2 * CANVAS_PADDING


# players
NO_PLAYER = ' '
PLAYER_1 = 'w'
PLAYER_2 = 'b'

# actions
ACTION_EXECUTE_MUEHLE = 'action_execute_muehle'
ACTION_STONE_SELECTED = 'action_stone_selected'

# color constants
BOARD_FOREGROUND_COLOR = '#191919'
BOARD_BACKGROUND_COLOR = '#ffffcb'
STASH_BACKGROUND_COLOR = '#dedede'
PLAYER_1_FILL_COLOR = '#E8E8E8'
PLAYER_1_STROKE_COLOR = '#191919'
PLAYER_2_FILL_COLOR = '#3D3D3D'
PLAYER_2_STROKE_COLOR = '#191919'
TEXT_COLOR = '#ffffff'

# shadow constants
SHADOW_COLOR = '#000000'
SHADOW_OFFSET_X = 2
SHADOW_OFFSET_Y = 2
SHADOW_BLUR = 2

# default shadow constants
SHADOW_COLOR_DEFAULT = 'rgba(0, 0, 0, 0)'
SHADOW_OFFSET_X_DEFAULT = 0
SHADOW_OFFSET_Y_DEFAULT = 0
SHADOW_BLUR_DEFAULT = 0


In [ ]:
# all available values for x and y, beginning left to right and top to bottom
av = [
    math.floor(CANVAS_PADDING),
    math.floor(CANVAS_PADDING + ROW_WIDTH),
    math.floor(CANVAS_PADDING + 2 * ROW_WIDTH),
    math.floor(BOARD_SIZE / 2),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + 2 * ROW_WIDTH)),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + ROW_WIDTH)),
    math.floor(BOARD_SIZE - CANVAS_PADDING)
]


In [ ]:
# All coordinates. The first tuple is the outer square, the second tuple is the middle square and the third tuple is the inner square. It always starts from top left and then clockwise.
coords = (
    (
        (av[0], av[0]),
        (av[3], av[0]),
        (av[6], av[0]),
        (av[6], av[3]),
        (av[6], av[6]),
        (av[3], av[6]),
        (av[0], av[6]),
        (av[0], av[3])
    ),
    (
        (av[1], av[1]),
        (av[3], av[1]),
        (av[5], av[1]),
        (av[5], av[3]),
        (av[5], av[5]),
        (av[3], av[5]),
        (av[1], av[5]),
        (av[1], av[3])
    ),
    (
        (av[2], av[2]),
        (av[3], av[2]),
        (av[4], av[2]),
        (av[4], av[3]),
        (av[4], av[4]),
        (av[3], av[4]),
        (av[2], av[4]),
        (av[2], av[3])
    )
) 

In [ ]:
# Function to draw a line on a canvas c, from start to end. Start and end are tuples with x and y values.
def draw_line(c, start, end):
    c.move_to(start[0], start[1])
    c.line_to(end[0], end[1])

In [ ]:
# Function to enable or disable shadows on a given canvas
def toggle_shadow(c, enable):
    c.shadow_color    = SHADOW_COLOR    if enable == True else SHADOW_COLOR_DEFAULT
    c.shadow_offset_x = SHADOW_OFFSET_X if enable == True else SHADOW_OFFSET_X_DEFAULT
    c.shadow_offset_y = SHADOW_OFFSET_Y if enable == True else SHADOW_OFFSET_Y_DEFAULT
    c.shadow_blur     = SHADOW_BLUR     if enable == True else SHADOW_BLUR_DEFAULT

In [ ]:
# Function to draw a circle on a canvas c with given coords, radius and color
def draw_circle(c, coords, radius, color, stroke_color = None, use_shadow = False, selected = False):
    if use_shadow == True:
        toggle_shadow(c, True)
    c.fill_style = color
    c.fill_arc(coords[0], coords[1], radius, 0, 2 * math.pi)
    if use_shadow == True:
        toggle_shadow(c, False)
    if stroke_color is not None:
        c.line_width = SELECTED_STONE_LINE_WIDTH if selected == True else DEFAULT_STONE_LINE_WIDTH
        c.stroke_color = stroke_color
        c.stroke_arc(coords[0], coords[1], radius, 0, 2 * math.pi)

In [ ]:
# Function to draw a (player) stone with given coords and for a given player (or the the default )
def draw_stone(c, coords, player, selected = False):
    if player == PLAYER_1:
        draw_circle(c, coords, PLAYER_PIECE_RADIUS, PLAYER_1_FILL_COLOR, stroke_color = PLAYER_1_STROKE_COLOR, use_shadow = True, selected = selected)
        draw_circle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), PLAYER_1_FILL_COLOR, stroke_color = PLAYER_1_STROKE_COLOR, selected = selected)
    elif player == PLAYER_2:
        draw_circle(c, coords, PLAYER_PIECE_RADIUS, PLAYER_2_FILL_COLOR, stroke_color = PLAYER_2_STROKE_COLOR, use_shadow = True, selected = selected)
        draw_circle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), PLAYER_2_FILL_COLOR, stroke_color = PLAYER_2_STROKE_COLOR, selected = selected)
    else:
        draw_circle(c, coords, DEFAULT_PIECE_RADIUS, BOARD_FOREGROUND_COLOR)

In [ ]:
def draw_text(c, text):
    c.clear()
    c.fill_text(text, TEXT_X, TEXT_Y, max_width = TEXT_MAX_WIDTH)

In [ ]:
# Function to construct a square on a canvas c.
def construct_square(c, square):
    for i in range(4):
        start = i * 2
        end = (i * 2 + 2) if (i * 2 + 2 <= 6) else 0 
        draw_line(c, square[start], square[end])

In [ ]:
# Function to construct the cross lines on a canvas c
def construct_cross_lines(c, coords):
    for i in range(4):
        k = i * 2 + 1
        draw_line(c, coords[0][k], coords[2][k])

In [ ]:
# Function to setup a canvas, returns a reference to a MultiCanvas
def setupCanvas():
    # level 0: Background and lines
    # level 1: Stones
    # level 2: Text
    canvas = ipycanvas.MultiCanvas(3, width = CANVAS_WIDTH, height = CANVAS_HEIGHT)
    canvas[0].fill_style = BOARD_BACKGROUND_COLOR
    canvas[0].fill_rect(0, 0, BOARD_SIZE, BOARD_SIZE)

    canvas[0].fill_style = STASH_BACKGROUND_COLOR
    canvas[0].fill_rect(STASH_STARTING_POINT_X, STASH_STARTING_POINT_Y, CANVAS_WIDTH, CANVAS_HEIGHT)


    canvas[0].stroke_style = BOARD_FOREGROUND_COLOR
    canvas[0].begin_path()

    for i in range(3):
        construct_square(canvas[0], coords[i])



    construct_cross_lines(canvas[0], coords)

    canvas[0].stroke()
    
    canvas[1].fill_style = TEXT_COLOR
    
    return canvas

In [ ]:
def update_gui(c, state, selectedStone = None):
    c.clear()
    ((stash_p1, stash_p2), squares) = state
    
    # update pieces on the board
    for i in range(len(squares)):
        for j in range(len(squares[i])):
            draw_stone(c, coords[i][j], squares[i][j], selected = ((selectedStone is not None) and (selectedStone[0] == i) and (selectedStone[1] ==j)))

    # update pieces on the stash

    # player 1

    x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
    y = STASH_STARTING_POINT_Y + CANVAS_PADDING + PLAYER_PIECE_RADIUS

    for i in range(stash_p1):
        x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
        draw_stone(c, (x, y), PLAYER_1)

    # player 2

    x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
    y = STASH_STARTING_POINT_Y + CANVAS_PADDING + 3 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING

    for i in range(stash_p2):
        x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
        draw_stone(c, (x, y), PLAYER_2)
    

## Game

In [ ]:
%run ./muehle-game.ipynb

In [ ]:
%run ./muehle-game-utils.ipynb

In [ ]:
# Test States
s1 = ((9, 9), (
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
))
s2 = ((0, 0), (
    ('w', 'w', ' ', 'b', 'w', 'b', 'w', 'w'),
    ('b', 'w', ' ', 'b', 'w', ' ', ' ', 'w'),
    ('b', 'b', 'b', 'b', ' ', ' ', 'b', 'w'),
))
s3 = ((7, 7), (
    ('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))
s4 = ((2, 8), (
    ('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))
s5 = ((0, 0), (('w', 'w', ' ', 'w', 'w', 'b', ' ', 'b'), (' ', 'b', 'w', ' ', 'w', 'w', 'b', 'b'), ('b', 'b', 'b', 'w', ' ', ' ', 'b', ' ')))

In [ ]:
def getClickedStone(x, y):
    for value in av:
        if value - PLAYER_PIECE_RADIUS <= x <= value + PLAYER_PIECE_RADIUS:
            logger.info(f'value for x = {x} found: {value}')
            x = value
        if value - PLAYER_PIECE_RADIUS <= y <= value + PLAYER_PIECE_RADIUS:
            logger.info(f'value for y = {y} found: {value}')
            y = value
    stone = None
    for i in range(len(coords)):
        if stone is not None:
            break
        for j in range(len(coords[i])):
            if coords[i][j] == (x,y):
                stone = (i, j)

    logger.info(f'stone: {stone}')
    return stone

In [ ]:
def convertStateTupleToList(s):
    return [list(s[0]), [list(s[1][0]), list(s[1][1]), list(s[1][2])]]

def convertStateListToTuple(s):
    return (tuple(s[0]), (tuple(s[1][0]), tuple(s[1][1]), tuple(s[1][2])))

In [ ]:
def checkIfStoneIsInMuehle(player, stone, state = None, muehlen = None):
    if muehlen is None and state is None:
        logger.error('muehlen or state must be provided!')
        raise('muehlen or state must be provided!')
    if muehlen is None:
        muehlen = findMuehlen(state[1], player)
    for muehle in muehlen:
        if stone in muehle:
            return True
    return False

In [ ]:
def checkIfAllStonesAreInMuehle(state, player):
    stones = findCellsOf(state[1], player)
    muehlen = findMuehlen(state[1], player)
    logger.info(f'allStones: {stones}')
    for stone in stones:
        if checkIfStoneIsInMuehle(player, stone, muehlen = muehlen) == False:
            return False
    return True

In [ ]:
class GameState:
    def __init__(self):
#         self.state = ((9, 9), (
#             (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
#             (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
#             (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
#         ))
        self.state = s1
        self.player = PLAYER_1
        self.canvas = setupCanvas()
        self.action = None
        self.muehlenLeft = 0
        self.selectedStone = None
        
        self.state_temp = None
        
        self.canvas[2].on_mouse_up(self.handle_mouse_up)
        
        update_gui(self.canvas[2], self.state)
        
        self.updateText()
        
        logger.info('game state initalized')
        
    def handle_mouse_up(self, x, y):
        logger.info(f'({x}, {y})')
        
        phase = playerPhase(self.state, self.player)
        logger.info(f'player phase: {phase}')
        
        stone = getClickedStone(x, y)
        
        if self.action == ACTION_EXECUTE_MUEHLE:
            self.selectMuehle(stone)
            return
        
        if self.action == ACTION_STONE_SELECTED:
            self.moveStoneTo(stone, jump = True if phase == 3 else False)
            return
        
        if phase == 1:
            self.playerPhaseOne(stone)
        else:
            if phase == 2 or phase == 3:
                self.selectStone(stone)
                    

    def playerPhaseOne(self, stone):
        if stone is None:
            return
        if self.state[1][stone[0]][stone[1]] != NO_PLAYER:
            logger.warning(f'{stone} is not free')
            return

        newState = convertStateTupleToList(self.state)

        newState[0][0 if self.player == PLAYER_1 else 1] -= 1#

        newState[1][stone[0]][stone[1]] = self.player

        allAvailableStates = nextStates(self.state, self.player)

        newState = convertStateListToTuple(newState)

        if self.validateNewState(newState) == True:
            logging.info('stone placed')

        else:
            logger.info('NewState not in allAvailableStates, checking for new Muehlen ...')
            newMuehlen = self.checkNewMuehlen(newState, phase = 1)

        self.updateText()
        
        
    # Player phase 2 and 3
    def selectStone(self, stone):
        if stone is None:
            return
        if self.state[1][stone[0]][stone[1]] != self.player:
            logger.warning(f'{stone} is not the own stone')
            return
        self.action = ACTION_STONE_SELECTED
        self.selectedStone = stone
        
        update_gui(self.canvas[2], self.state, selectedStone = self.selectedStone)
        
        self.updateText()
            
    def checkNewMuehlen(self, newState, phase = None):
        oldMuehlen = findMuehlen(self.state[1], self.player)
        newMuehlen = countNewMuehlen(newState[1], oldMuehlen, self.player)

        if newMuehlen > 0:
            self.state_temp = newState
            self.action = ACTION_EXECUTE_MUEHLE
            self.muehlenLeft = newMuehlen if phase != 1 else 1
            
            update_gui(self.canvas[2], self.state_temp)
    
        return newMuehlen
    
    def selectMuehle(self, stone):
        if self.muehlenLeft <= 0:
            logger.warning('No Muehlen left!')
            return
        if stone is None:
            return
        if self.state[1][stone[0]][stone[1]] != opponent(self.player):
            logger.warning(f'{stone} is not the opponent!')
            return
        
        # check if stone is in muehle:
        stoneIsInMuehle = checkIfStoneIsInMuehle(opponent(self.player), stone, state = self.state_temp)
        logger.info(f'{stone} is in muehle: {stoneIsInMuehle}')
        if stoneIsInMuehle == True:
            allStonesAreInMuehle = checkIfAllStonesAreInMuehle(self.state_temp, opponent(self.player))
            logger.info(f'checkIfAllStonesAreInMuehle: {allStonesAreInMuehle}')
            if allStonesAreInMuehle == False:
                return
                
        newState = convertStateTupleToList(self.state_temp)
        newState[1][stone[0]][stone[1]] = NO_PLAYER
        newState = convertStateListToTuple(newState)

        if self.muehlenLeft == 1:
            if self.validateNewState(newState):
                logger.info('success')
            else:
                logger.critical('STRANGE BEHAVIOR')    
        else:
            self.muehlenLeft -= 1
            self.state_temp = newState
            self.action = ACTION_EXECUTE_MUEHLE
            update_gui(self.canvas[2], self.state_temp)
        
        self.updateText()
    
    def togglePlayer(self):
        self.player = PLAYER_1 if self.player == PLAYER_2 else PLAYER_2
        
        
        
    def validateNewState(self, newState):
        
        allAvailableStates = nextStates(self.state, self.player)
        if newState in allAvailableStates:
            logger.info('SUCCESS')

            self.action = None
            self.state = newState
            self.state_temp = None
            self.muehlenLeft = 0
            self.selectedStone = None

            self.togglePlayer()

            update_gui(self.canvas[2], self.state)
            
            return True
        else:
            logger.critical('state is not in allAvailableStates!')
            return False
        
        
    def moveStoneTo(self, coord, jump = False):
        logger.info(f'moveStoneTo({coord}, jump = {jump})')
        if coord is None:
            return
        if self.state[1][coord[0]][coord[1]] != NO_PLAYER:
            logger.warning(f'{coord} is not free')
            return

        if jump == True or coord in findNeighboringEmptyCells(self.state[1], self.selectedStone):
            
            newState = convertStateTupleToList(self.state)
            newState[1][self.selectedStone[0]][self.selectedStone[1]] = NO_PLAYER
            newState[1][coord[0]][coord[1]] = self.player
            newState = convertStateListToTuple(newState)
            
            logger.info(newState)
            
            if self.validateNewState(newState) == True:
                movement = 'jumped' if jump == True else 'moved'
                logger.info(f'Stone successfully {movement}!')
            else:
                logger.info('Round not finished, checking for new muehlen...')
                self.checkNewMuehlen(newState)
        else:
            logger.error(f'{coord} is not a (free) neighbor of {self.selectedStone}!')
        
        self.updateText()
            
    def updateText(self):
        phase = playerPhase(self.state, self.player)
        message = f'Player {self.player}: '

        if self.action is not None:
            if self.action == ACTION_EXECUTE_MUEHLE:
                message += 'execute your Muehle.'
            else:
                if self.action == ACTION_STONE_SELECTED:
                    movement = 'jump' if jump == True else 'move'
                    message += f'{movement} your selected stone'
        else:
            if phase == 1:
                message += 'Place your stone.'
            else:
                if phase == 2 or phase == 3:
                    movement = 'jump' if jump == True else 'move'
                    message += f'select your stone you want to {movement}'
        
        logging.info(message)
        draw_text(self.canvas[1], message)
        

In [ ]:
gameState = GameState()

In [ ]:
gameState.canvas